In [2]:
%load_ext autoreload
%autoreload 2

import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader



from config import *
from utils.eda_functions import (load_from_pickle, save_to_pickle)

In [3]:
X_train = load_from_pickle(os.path.join(DATA_DIR, 'Vitals_train.pkl'))
X_dev = load_from_pickle(os.path.join(DATA_DIR, 'Vitals_dev.pkl'))
X_test = load_from_pickle(os.path.join(DATA_DIR, 'Vitals_test.pkl'))

Ys_train = load_from_pickle(os.path.join(DATA_DIR, 'Y_train.pkl'))
Ys_dev = load_from_pickle(os.path.join(DATA_DIR, 'Y_dev.pkl'))
Ys_test = load_from_pickle(os.path.join(DATA_DIR, 'Y_test.pkl'))

In [7]:
Y_train, Y_dev, Y_test = Ys_train['mort_hosp'], Ys_dev['mort_hosp'], Ys_test['mort_hosp']

In [6]:
Ys_train.drop(columns=['mort_icu'], inplace=True)

In [8]:
# Custom Dataset
class MortalityDataset(Dataset):
    def __init__(self, X_df, Y_df):
        self.X = torch.tensor(X_df.values, dtype=torch.float32)
        self.Y = torch.tensor(Y_df.values, dtype=torch.long)  # Assuming labels are integers
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, index):
        x = self.X[index]
        y = self.Y[index]
        return x, y


In [9]:
# Custom GRU Cell for GRU-D
class GRUDCell(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(GRUDCell, self).__init__()
        
        self.hidden_size = hidden_size
        self.zl = nn.Linear(input_size + hidden_size, hidden_size)
        self.rl = nn.Linear(input_size + hidden_size, hidden_size)
        self.hl = nn.Linear(input_size + hidden_size, hidden_size)
    
    def forward(self, x, h, mask):
        combined = torch.cat((x, h, mask), 1)
        z = torch.sigmoid(self.zl(combined))
        r = torch.sigmoid(self.rl(combined))
        
        combined_r = torch.cat((x, r * h, mask), 1)
        h_tilda = torch.tanh(self.hl(combined_r))
        
        h_out = (1 - z) * h + z * h_tilda
        return h_out


In [10]:
class GRUDModel(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(GRUDModel, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.grud_cell = GRUDCell(input_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, 1)
    
    def forward(self, x, mask):
        h = torch.zeros(x.size(0), self.hidden_dim)
        for t in range(x.size(1)):
            h = self.grud_cell(x[:, t, :], h, mask[:, t, :])
        
        out = self.fc(h)
        out = torch.sigmoid(out)
        return out


In [11]:
# Assuming X_train and Y_train are pandas DataFrames
train_dataset = MortalityDataset(X_train, Y_train)

# Create DataLoader
batch_size = 64  # You can change this according to your needs
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)


In [12]:
# Initialize the model, loss, and optimizer
model = GRUDModel(input_dim=264, hidden_dim=128)  # Initialize the model
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Using Adam optimizer
# Class weights can be adjusted based on the imbalance ratio.
class_weights = torch.tensor([0.1, 0.9])
criterion = nn.CrossEntropyLoss(weight=class_weights)


In [13]:
model

GRUDModel(
  (grud_cell): GRUDCell(
    (zl): Linear(in_features=392, out_features=128, bias=True)
    (rl): Linear(in_features=392, out_features=128, bias=True)
    (hl): Linear(in_features=392, out_features=128, bias=True)
  )
  (fc): Linear(in_features=128, out_features=1, bias=True)
)

In [14]:
# Hyperparameters
learning_rate = 0.001
num_epochs = 10

In [15]:
# Training Loop
for epoch in range(num_epochs):
    for i, (x, y) in enumerate(train_loader):
        x = Variable(x)
        y = Variable(y)
        
        # Forward pass
        outputs = model(x)
        loss = criterion(outputs, y)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')

print('Finished Training')


IndexError: index 211406 is out of bounds for dimension 0 with size 16760